Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [144]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [145]:
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer

In [146]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [147]:
import numpy as np

# This will create a wrangle function for our data

def wrangle(X):
  X = X.copy()

  # This will convert 'date_recorded' to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  # This will extract the components from 'date_recorded', then drop the now redundant
  # original column
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')
  # This will clean small latitude values
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  # This will engineer a new feature: Water used per person 'water_per_person'
  X['water_per_person'] = X['amount_tsh'].replace({0:1})/X['population'].replace({0:1})
  # This will engineer a new feature: Age of pump 'years_old' and clean the
  # 'construction_year' column
  X['years_old'] = X['year_recorded']-X['construction_year'].replace({0:1990})
  # This will engineer a new feature: average population served per year
  # 'served_per_year'
  X['served_per_year'] = X['population'].replace({0:1})/X['years_old'].replace({0:1})
  # This will replace remaining zeros with nulls, so that we can impute them later on
  # This will also establish a "missing" indicator, which may be used for predictive
  # purposes later on as well
  cols_with_zeros = ['longitude', 'latitude', 'gps_height']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
    X[col+'_MISSING'] = X[col].isnull()
  # This will wash out duplicate or redundant columns
  duplicates = ['quantity_group', 'payment_type']
  X = X.drop(columns=duplicates)
  # This will drop values that remain static or always vary, neither of which is
  # useful for predictive purposes
  unusable_variance = ['recorded_by']
  X = X.drop(columns=unusable_variance)
  # This will set index to 'id'
  X.set_index(X['id'], inplace=True)

  # This will return the modified dataframe
  return X

In [148]:
df=wrangle(train)
test=wrangle(test)

In [149]:
X_train = df.drop(['status_group'], axis=1)
y_train = pd.DataFrame(df['status_group'])

In [150]:
y_train.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


# Let's try a different approach from the last few days

 * Let's check for cardinality, then split our data from there
 * This should let us use both OneHot and Ordinal encoding for each
 * Afterwards, we should be able to merge them together, & pass it through the pipeline, and cross-validate.

In [151]:
# Let's create a cardinality check function

def cardinality_check (n, df):
  features = []
  cardinality_score = []

  for _ in range(len(df.columns)):
    if len(df[df.columns[_]].value_counts()) > n:
      features.append(df.columns[_])
      cardinality_score.append(len(df[df.columns[_]].value_counts()))

  feature_list = dict(zip(features, cardinality_score))

  return feature_list

In [152]:
# This will call our function and allow us find a high cardinality cutoff point
high_cardinality = cardinality_check(150, X_train)
# This will repeat the process for our test data
high_cardinality_test = cardinality_check(150, test)

In [153]:
# This will create a dataframe, using the results of our function, of LOW cardinality features
lc_X_train = X_train.drop(columns= list(high_cardinality.keys()))
# This will repeat the process for our test data
lc_X_test = test.drop(columns= list(high_cardinality_test.keys()))

In [154]:
# This will create a dataframe of HIGH cardinality features
hc_X_train = X_train[list(high_cardinality.keys())]
# This will repeat the process for our test data
hc_X_test = test[list(high_cardinality.keys())]

In [155]:
one_hot = ce.OneHotEncoder(use_cat_names=True)
one_hot.fit(lc_X_train, y_train['status_group'])
lc_X_train = one_hot.transform(lc_X_train)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [156]:
lc_X_test = one_hot.transform(lc_X_test)

In [157]:
ordinal = ce.OrdinalEncoder()
ordinal.fit(hc_X_train, y_train['status_group'])
hc_X_train = ordinal.transform(hc_X_train)

In [158]:
hc_X_test = ordinal.transform(hc_X_test)

In [159]:
X_train2 = lc_X_train.merge(hc_X_train, on=lc_X_train.index)

In [160]:
X_test2 = lc_X_test.merge(hc_X_test, on=lc_X_test.index)

In [161]:
impute = SimpleImputer(missing_values=np.nan, strategy='mean')
impute.fit(X_train2, y_train['status_group'])
X_test2 = impute.transform(X_test2)

In [165]:
X_test2

array([[5.07850000e+04, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 3.11526480e-03, 3.21000000e+02],
       [5.16300000e+04, 0.00000000e+00, 0.00000000e+00, ...,
        4.17000000e+02, 3.33333333e-03, 2.30769231e+01],
       [1.71680000e+04, 0.00000000e+00, 0.00000000e+00, ...,
        9.38000000e+02, 2.00000000e-03, 1.66666667e+02],
       ...,
       [2.87490000e+04, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 5.00000000e-03, 6.66666667e+01],
       [3.34920000e+04, 0.00000000e+00, 0.00000000e+00, ...,
        1.04400000e+03, 6.66666667e-03, 3.75000000e+01],
       [6.87070000e+04, 0.00000000e+00, 0.00000000e+00, ...,
        1.16000000e+02, 2.50000000e-02, 8.00000000e+00]])

In [19]:
# low_card = list(set(X_train.columns).difference(set(high_cardinality.keys())))
# print(high_cardinality.keys())

In [20]:
# X_train.select_dtypes('object').nunique().sort_values()

In [21]:
# ct = ColumnTransformer([ 
#                        ('ohe', OneHotEncoder(use_cat_names=True), list(high_cardinality.keys())),
#                        ('ordinal', OrdinalEncoder(), low_card)
# ])

# model = make_pipeline(
#    ct,
#    SimpleImputer(),
#    RandomForestClassifier()
# )

#rf.fit(X_train, y_train)

In [22]:
# model.fit(X_train, y_train)

In [23]:
# from sklearn import set_config
# set_config(display='diagram')
# model

# Now that we have our modified training data, let's establish a baseline

In [24]:
print('Baseline Accuracy is:', df['status_group'].value_counts(normalize=True).max())

Baseline Accuracy is: 0.543080808080808


# Let's build that model!

In [25]:
rf = make_pipeline(
    SimpleImputer(),
    RandomForestClassifier()
)

In [26]:
rf_cvs = cross_val_score(rf, X_train2, y_train, cv=5, n_jobs=-1)

In [27]:
print(rf_cvs)
print(rf_cvs.mean())
print(rf_cvs.std())

[0.81557239 0.80757576 0.81430976 0.81111111 0.80900673]
0.8115151515151515
0.0030412261045100314


In [34]:
parasect = {'randomforestclassifier__n_estimators': np.arange(20, 61, 20),
         'randomforestclassifier__max_depth': np.arange(10, 31, 10),
         'randomforestclassifier__max_samples': np.arange(0.3, 0.71, 0.2)}

In [35]:
rf_gs = GridSearchCV(rf, parasect, cv=5, n_jobs=-1, verbose=1)

In [36]:
rf_gs.fit(X_train2, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 12.2min finished
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('simpleimputer',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
                                                      strategy='mean',
                                                      verbose=0)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                     

In [37]:
pd.DataFrame(rf_gs.cv_results_).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
mean_fit_time,5.08839,6.59831,7.81148,5.58921,7.17851,9.47306,5.7981,8.36837,10.7694,5.80597,8.50649,11.163,7.1776,10.9978,15.2064,8.11663,12.8212,17.1996,6.39737,9.26601,12.0287,7.70976,12.286,16.3171,8.95077,14.6901,18.9971
std_fit_time,0.0475779,0.164146,0.0485897,0.331603,0.0596261,0.0939136,0.0874622,0.084722,0.0913886,0.088532,0.0877911,0.0659265,0.0473473,0.0742625,0.287301,0.0916085,0.0396547,0.102009,0.081504,0.21281,0.13964,0.057456,0.456643,0.108441,0.200787,0.294198,1.28723
mean_score_time,0.519388,0.618979,0.680754,0.500821,0.585128,0.695703,0.51846,0.587565,0.675882,0.598911,0.706625,0.877566,0.590843,0.740516,0.995281,0.60538,0.764468,0.930419,0.605469,0.783531,0.965208,0.591171,0.80597,0.990371,0.641724,0.820203,0.949274
std_score_time,0.0241217,0.0358025,0.0247749,0.013521,0.0152694,0.0123574,0.0195576,0.0131609,0.0262943,0.0458693,0.0383571,0.0424886,0.019562,0.0294262,0.0668649,0.0327219,0.0236893,0.0122903,0.0359967,0.0240848,0.0386652,0.0199124,0.043594,0.0250197,0.0253725,0.0182081,0.131873
param_randomforestclassifier__max_depth,10,10,10,10,10,10,10,10,10,20,20,20,20,20,20,20,20,20,30,30,30,30,30,30,30,30,30
param_randomforestclassifier__max_samples,0.3,0.3,0.3,0.5,0.5,0.5,0.7,0.7,0.7,0.3,0.3,0.3,0.5,0.5,0.5,0.7,0.7,0.7,0.3,0.3,0.3,0.5,0.5,0.5,0.7,0.7,0.7
param_randomforestclassifier__n_estimators,20,40,60,20,40,60,20,40,60,20,40,60,20,40,60,20,40,60,20,40,60,20,40,60,20,40,60
params,"{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran...","{'randomforestclassifier__max_depth': 30, 'ran..."
split0_test_score,0.748822,0.749579,0.752104,0.747475,0.752189,0.751936,0.753283,0.748485,0.752441,0.792424,0.802189,0.798401,0.798148,0.802778,0.805556,0.800421,0.805387,0.806818,0.796886,0.805724,0.80303,0.805471,0.809933,0.809933,0.80766,0.811532,0.813721
split1_test_score,0.743013,0.744444,0.747138,0.745875,0.74596,0.744444,0.749158,0.747054,0.74697,0.792088,0.794192,0.795034,0.793434,0.799663,0.8,0.801515,0.801431,0.804209,0.791077,0.796044,0.802609,0.799327,0.80362,0.807997,0.798737,0.806902,0.806145


In [38]:
rf_gs.best_estimator_

Pipeline(memory=None,
         steps=[('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=30, max_features='auto',
                                        max_leaf_nodes=None, max_samples=0.7,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=60, n_jobs=None,
                                        oob_score=False

In [39]:
rf_gs.best_score_

0.8096632996632996

In [40]:
rf_gs.best_params_

{'randomforestclassifier__max_depth': 30,
 'randomforestclassifier__max_samples': 0.7,
 'randomforestclassifier__n_estimators': 60}

In [41]:
rf_rs = RandomizedSearchCV(rf, param_distributions=parasect, n_iter=3, cv=5, n_jobs=-1, verbose=1)

In [42]:
rf_rs.fit(X_train2, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,
                                                            missing_values=nan,
                                                            strategy='mean',
                                                            verbose=0)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(bootstrap=True,
                                                                     ccp_alpha=0.0,
                                                                     class_weight=None,
                                                        

In [43]:
rf_rs.best_score_

0.8093602693602694

In [97]:
X_test.head()

,amount_tsh,num_private,basin_Lake Nyasa,basin_Lake Victoria,basin_Pangani,basin_Ruvuma / Southern Coast,basin_Internal,basin_Lake Tanganyika,basin_Wami / Ruvu,basin_Rufiji,basin_Lake Rukwa,region_Iringa,region_Mara,region_Manyara,region_Mtwara,region_Kagera,region_Tanga,region_Shinyanga,region_Tabora,region_Pwani,region_Ruvuma,region_Kilimanjaro,region_Rukwa,region_Mwanza,region_Kigoma,region_Lindi,region_Dodoma,region_Arusha,region_Mbeya,region_Singida,region_Morogoro,region_Dar es Salaam,region_code,district_code,lga_Ludewa,lga_Serengeti,lga_Simanjiro,lga_Nanyumbu,lga_Karagwe,lga_Mkinga,...,source_type_dam,source_type_borehole,source_type_other,source_type_shallow well,source_type_river/lake,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_hand pump,waterpoint_type_other,waterpoint_type_improved spring,waterpoint_type_cattle trough,waterpoint_type_dam,waterpoint_type_group_communal standpipe,waterpoint_type_group_hand pump,waterpoint_type_group_other,waterpoint_type_group_improved spring,waterpoint_type_group_cattle trough,waterpoint_type_group_dam,year_recorded,month_recorded,day_recorded,years_old,longitude_MISSING,latitude_MISSING,gps_height_MISSING,funder,gps_height,installer,longitude,latitude,wpt_name,subvillage,ward,population,scheme_name,water_per_person,served_per_year
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,3,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,2013,2,4,1,False,False,False,164.0,1996.0,342.0,35.290799,-4.059696,-1.0,10944.0,574.0,321,2.0,0.003115,321.000000
51630,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,2013,2,4,13,False,False,False,21.0,1569.0,6.0,36.656709,-3.309214,-1.0,-1.0,368.0,300,417.0,0.003333,23.076923
17168,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,13,2,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,2013,2,1,3,False,False,False,26.0,1567.0,20.0,34.767863,-5.004344,21519.0,7345.0,648.0,500,938.0,0.002000,166.666667
45559,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,80,43,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,2013,1,22,26,False,False,False,145.0,267.0,131.0,38.058046,-9.418672,-1.0,5580.0,1796.0,250,2.0,0.004000,9.615385
49871,500.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,10,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,2013,3,27,13,False,False,False,1038.0,1260.0,1133.0,35.006123,-10.950412,2985.0,2891.0,654.0,60,319.0,8.333333,4.615385


In [99]:
X_test.shape

(14358, 312)

In [125]:
y_pred = rf_rs.predict(X_test)

In [68]:
y_pred

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [167]:
data = {'id': test.index, 'status_group': y_pred}
submission = pd.DataFrame(data)

In [168]:
submission.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [170]:
submission.set_index('id', inplace=True)

In [171]:
submission.head()

,status_group
id,
50785,non functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [172]:
submission.to_csv('Submission3.csv')